In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from scipy.stats import genpareto
from sklearn.ensemble import IsolationForest  # Using Isolation Forest for anomaly detection
!pip install keras-tuner
import kerastuner as kt  # make sure to install keras-tuner (pip install keras-tuner)

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# =============================================================================
# STEP 1: Load and Split the Dataset
# =============================================================================
# Load the full training data (Normal Condition)
X_train_full = pd.read_excel(io="Freq_Z24N.xlsx", sheet_name="Train Set", header=0, engine="openpyxl").values

# Split the training set into training (80%) and validation (20%) sets
X_train, X_val = train_test_split(X_train_full, test_size=0.2, random_state=42)

# Load the test set, which remains unseen. It contains two parts:
#   - first part: Inspection - Normal Condition (samples 1 to 510)
#   - second part: Inspection - Damaged Condition (samples 511 to end)
X_test_full = pd.read_excel(io="Freq_Z24N.xlsx", sheet_name="Test Set", header=0, engine="openpyxl").values
X_test_normal = X_test_full[:510]    # Inspection - Normal Condition
X_test_damaged = X_test_full[510:]     # Inspection - Damaged Condition

# Standardize features using training data statistics (from X_train only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_normal_scaled = scaler.transform(X_test_normal)
X_test_damaged_scaled = scaler.transform(X_test_damaged)

input_dim = X_train_scaled.shape[1]

# =============================================================================
# STEP 2: Define a Model Builder for Hyperparameter Tuning (VAE)
# =============================================================================
def build_vae(hp):
    # Choose hyperparameters
    latent_dim = hp.Choice('latent_dim', values=[8, 16, 32])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    l2_reg = hp.Float('l2_reg', min_value=1e-4, max_value=1e-2, sampling='LOG')
    beta = hp.Float('beta', min_value=0.05, max_value=0.5, step=0.05)

    # Encoder
    encoder_inputs = layers.Input(shape=(input_dim,), name="encoder_input")
    x = layers.Dense(256, activation='relu', kernel_regularizer=l2(l2_reg))(encoder_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    # "z_mean" will be used later as the latent representation.
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    z = layers.Lambda(sampling, name="z")([z_mean, z_log_var])

    # Decoder
    decoder_inputs = layers.Input(shape=(latent_dim,), name="decoder_input")
    x_dec = layers.Dense(64, activation='relu', kernel_regularizer=l2(l2_reg))(decoder_inputs)
    x_dec = layers.BatchNormalization()(x_dec)
    x_dec = layers.Dropout(dropout_rate)(x_dec)
    x_dec = layers.Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(x_dec)
    x_dec = layers.BatchNormalization()(x_dec)
    x_dec = layers.Dropout(dropout_rate)(x_dec)
    x_dec = layers.Dense(256, activation='relu', kernel_regularizer=l2(l2_reg))(x_dec)
    x_dec = layers.BatchNormalization()(x_dec)
    x_dec = layers.Dropout(dropout_rate)(x_dec)
    decoder_outputs = layers.Dense(input_dim, activation='linear')(x_dec)

    # Build decoder model (used for prediction later)
    decoder = Model(decoder_inputs, decoder_outputs, name="decoder")

    # Custom loss layer
    class VAELossLayer(layers.Layer):
        def __init__(self, beta=beta, **kwargs):
            super().__init__(**kwargs)
            self.beta = beta

        def call(self, inputs):
            x, x_decoded, z_mean, z_log_var = inputs
            reconstruction_loss = tf.reduce_mean(tf.square(x - x_decoded), axis=1) * input_dim
            kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            total_loss = tf.reduce_mean(reconstruction_loss + self.beta * kl_loss)
            self.add_loss(total_loss)
            return x_decoded

    vae_outputs = decoder(z)
    vae_loss_output = VAELossLayer(name='vae_loss')([encoder_inputs, vae_outputs, z_mean, z_log_var])
    vae = Model(encoder_inputs, vae_loss_output, name="vae")
    vae.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=hp.Float('learning_rate', 1e-4, 1e-3, sampling='LOG')
    ))
    return vae

# =============================================================================
# STEP 3: Hyperparameter Tuning Using Keras Tuner (VAE)
# =============================================================================
tuner = kt.RandomSearch(
    build_vae,
    objective='val_loss',
    max_trials=10,  # Adjust the number of trials as needed
    executions_per_trial=1,
    directory='vae_tuner_dir',
    project_name='anomaly_detection_tuning'
)

# Use the 20% validation split (from the training set) for tuning
tuner.search(X_train_scaled, X_train_scaled,
             epochs=100,
             batch_size=64,
             validation_data=(X_val_scaled, X_val_scaled),
             callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, min_delta=0.001)],
             verbose=1)

# Retrieve the best model and hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
print(f"  latent_dim: {best_hps.get('latent_dim')}")
print(f"  dropout_rate: {best_hps.get('dropout_rate')}")
print(f"  l2_reg: {best_hps.get('l2_reg')}")
print(f"  beta: {best_hps.get('beta')}")
print(f"  learning_rate: {best_hps.get('learning_rate')}")

# Build the VAE model with best hyperparameters and train it
best_model = build_vae(best_hps)
early_stop = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True, min_delta=0.001)
history = best_model.fit(X_train_scaled, X_train_scaled,
                         epochs=500,
                         batch_size=64,
                         validation_data=(X_val_scaled, X_val_scaled),
                         callbacks=[early_stop],
                         verbose=1)

# =============================================================================
# STEP 4: Extract Latent Representations Using the Trained VAE Encoder
# =============================================================================
encoder = Model(inputs=best_model.input, outputs=best_model.get_layer("z_mean").output)

latent_train = encoder.predict(X_train_scaled)
latent_val = encoder.predict(X_val_scaled)
latent_test_normal = encoder.predict(X_test_normal_scaled)
latent_test_damaged = encoder.predict(X_test_damaged_scaled)

# =============================================================================
# STEP 5: Isolation Forest Model on the Latent Representations
# =============================================================================
# Define a parameter grid for Isolation Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_samples': [0.5, 0.8, 'auto'],
    'contamination': [0.01, 0.05, 0.1]
}

best_if = None
best_outlier_fraction = np.inf
best_params = {}

for n_estimators in param_grid['n_estimators']:
    for max_samples in param_grid['max_samples']:
        for contamination in param_grid['contamination']:
            if_model = IsolationForest(n_estimators=n_estimators,
                                       max_samples=max_samples,
                                       contamination=contamination,
                                       random_state=42)
            if_model.fit(latent_train)
            preds = if_model.predict(latent_train)  # +1 for inliers, -1 for outliers
            outlier_fraction = np.mean(preds == -1)
            if outlier_fraction < best_outlier_fraction:
                best_outlier_fraction = outlier_fraction
                best_params = {'n_estimators': n_estimators, 'max_samples': max_samples, 'contamination': contamination}
                best_if = if_model

print("Best Isolation Forest parameters found:")
print(best_params)
print(f"Fraction of outliers on training data: {best_outlier_fraction:.4f}")

# =============================================================================
# STEP 6: Compute Anomaly Scores Using the Isolation Forest Model Only
# =============================================================================
# The decision_function returns a score where higher values indicate normality.
# We convert these to anomaly measures by taking the negative, so that higher values indicate more anomalous.
train_scores = best_if.decision_function(latent_train)
val_scores = best_if.decision_function(latent_val)
test_normal_scores = best_if.decision_function(latent_test_normal)
test_damaged_scores = best_if.decision_function(latent_test_damaged)

train_if_anomaly = -train_scores
val_if_anomaly = -val_scores
test_normal_if_anomaly = -test_normal_scores
test_damaged_if_anomaly = -test_damaged_scores

# Normalize Isolation Forest anomaly scores using training data statistics
min_train_if = np.min(train_if_anomaly)
max_train_if = np.max(train_if_anomaly)
norm_train_if = (train_if_anomaly - min_train_if) / (max_train_if - min_train_if)
norm_val_if = (val_if_anomaly - min_train_if) / (max_train_if - min_train_if)
norm_test_normal_if = (test_normal_if_anomaly - min_train_if) / (max_train_if - min_train_if)
norm_test_damaged_if = (test_damaged_if_anomaly - min_train_if) / (max_train_if - min_train_if)

# =============================================================================
# (Optional) STEP 7: EVT Calibration on Isolation Forest Anomaly Scores
# =============================================================================
# Here we set a threshold using Extreme Value Theory (EVT) on the normalized anomaly scores.
u = np.percentile(norm_train_if, 90)
excesses = norm_train_if[norm_train_if > u] - u
params = genpareto.fit(excesses)
tail_probability = 0.95  # Adjust as desired
threshold_evt = u + genpareto.ppf(tail_probability, *params)
print(f"Final EVT-calibrated Isolation Forest threshold: {threshold_evt:.4f}")

# For your final anomaly score, we will use the normalized Isolation Forest anomaly scores directly.
final_train_scores = norm_train_if
final_val_scores = norm_val_if
final_test_normal_scores = norm_test_normal_if
final_test_damaged_scores = norm_test_damaged_if

# =============================================================================
# STEP 8: Combine Isolation Forest Anomaly Scores into a Single DataFrame
# =============================================================================
df_train = pd.DataFrame({
    "IF_Anomaly_Score": final_train_scores,
    "Dataset": "Training - Normal"
})
df_val = pd.DataFrame({
    "IF_Anomaly_Score": final_val_scores,
    "Dataset": "Validation - Normal"
})
df_test_normal = pd.DataFrame({
    "IF_Anomaly_Score": final_test_normal_scores,
    "Dataset": "Inspection - Normal"
})
df_test_damaged = pd.DataFrame({
    "IF_Anomaly_Score": final_test_damaged_scores,
    "Dataset": "Inspection - Damaged"
})
all_scores_df = pd.concat([df_train, df_val, df_test_normal, df_test_damaged], ignore_index=True)

summary_df = pd.DataFrame({
    "Mean_Train_IF_Score": [np.mean(final_train_scores)],
    "Std_Train_IF_Score": [np.std(final_train_scores)],
    "Mean_Val_IF_Score": [np.mean(final_val_scores)],
    "Std_Val_IF_Score": [np.std(final_val_scores)],
    "Mean_Test_Normal_IF_Score": [np.mean(final_test_normal_scores)],
    "Std_Test_Normal_IF_Score": [np.std(final_test_normal_scores)],
    "Mean_Test_Damaged_IF_Score": [np.mean(final_test_damaged_scores)],
    "Std_Test_Damaged_IF_Score": [np.std(final_test_damaged_scores)]
})

# =============================================================================
# STEP 9: Save Results to an Excel File with Two Sheets
# =============================================================================
output_excel_file = "IF_AnomalyDetection_Results.xlsx"
with pd.ExcelWriter(output_excel_file) as writer:
    all_scores_df.to_excel(writer, sheet_name="IF_Anomaly_Scores", index=False)
    summary_df.to_excel(writer, sheet_name="IF_Score_Summary", index=False)

print(f"Results saved to '{output_excel_file}'.")

# =============================================================================
# STEP 10: Visualization (Optional)
# =============================================================================
plt.figure(figsize=(12, 6))
plt.scatter(range(len(final_train_scores)), final_train_scores, s=3, label="Training - Normal")
plt.scatter(len(final_train_scores) + np.arange(len(final_val_scores)), final_val_scores,
            s=3, c='green', label="Validation - Normal")
plt.scatter(len(final_train_scores) + len(final_val_scores) + np.arange(len(final_test_normal_scores)),
            final_test_normal_scores, s=3, c='orange', label="Inspection - Normal")
plt.scatter(len(final_train_scores) + len(final_val_scores) + len(final_test_normal_scores) + np.arange(len(final_test_damaged_scores)),
            final_test_damaged_scores, s=3, c='red', label="Inspection - Damaged")
# Plot the EVT-calibrated threshold.
plt.axhline(threshold_evt, c='black', linestyle='--', label="EVT Threshold")
plt.ylabel("Normalized Isolation Forest Anomaly Score")
plt.xlabel("Sample Index")
plt.title("Isolation Forest-based Anomaly Detection")
plt.legend()
plt.show()
